Reference:
* https://github.com/fede1024/caffe-experiments/blob/master/memo.txt
* https://github.com/TZstatsADS/Spr2016-Proj3-Grp3/blob/master/output/extractfeature.ipynb 
* https://cdn.rawgit.com/TZstatsADS/ADS_Teaching/master/Spring2016/Tutorials/wk7-image_analysis/advanced_image_analysis.html

### 0. Input

* caffepath indicates the root path of the `caffe` package
* inputpath indicates the folder which saves all the training images
* inputpath_test indicates the folder which saves all the testing images
* outputpath indicates the foler which the features extracted should be saved in

In [2]:
caffepath = '/Users/YaqingXie/caffe'
inputpath = '/Users/YaqingXie/Desktop/3-Applied Data Science/Project3_poodleKFC_train/images'
inputpath_test = '/Users/YaqingXie/Desktop/image_new'
outputpath = '/Users/YaqingXie/Desktop'

### 1. Setup

* First, set up Python, `numpy`, `panda`, `datetime` and `matplotlib`.

In [3]:
# set up Python environment: numpy for numerical routines, and matplotlib for plotting
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
# display plots in this notebook
%matplotlib inline

# set display defaults
plt.rcParams['figure.figsize'] = (10, 10)        # large images
plt.rcParams['image.interpolation'] = 'nearest'  # don't interpolate: show square pixels
plt.rcParams['image.cmap'] = 'gray'  # use grayscale output rather than a (potentially misleading) color heatmap

* Load `caffe`.

In [4]:
# The caffe module needs to be on the Python path;
#  we'll add it here explicitly.
import sys

if not caffepath.endswith('/'):
    caffepath = caffepath + '/'
if not inputpath.endswith('/'):
    inputpath = inputpath + '/'
if not inputpath_test.endswith('/'):
    inputpath_test = inputpath_test + '/'
if not outputpath.endswith('/'):
    outputpath = outputpath + '/'

caffe_root = caffepath
sys.path.insert(0, caffe_root + 'python')

import caffe

* If needed, download the reference model ("CaffeNet", a variant of AlexNet).

In [5]:
import os
if os.path.isfile(caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'):
    print('CaffeNet found.')
else:
    print('Downloading pre-trained CaffeNet model...')
    !../scripts/download_model_binary.py ../models/bvlc_reference_caffenet

CaffeNet found.


### 2. Load net and set up input preprocessing

* Set Caffe to CPU mode and load the net from disk.

In [6]:
caffe.set_mode_cpu()

model_def = caffe_root + 'models/bvlc_reference_caffenet/deploy.prototxt'
model_weights = caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'

net = caffe.Net(model_def,      # defines the structure of the model
                model_weights,  # contains the trained weights
                caffe.TEST)     # use test mode (e.g., don't perform dropout)

* Set up input preprocessing. (We'll use Caffe's `caffe.io.Transformer` to do this, but this step is independent of other parts of Caffe, so any custom preprocessing code may be used).

    Our default CaffeNet is configured to take images in BGR format. Values are expected to start in the range [0, 255] and then have the mean ImageNet pixel value subtracted from them. In addition, the channel dimension is expected as the first (_outermost_) dimension.
    
    As matplotlib will load images with values in the range [0, 1] in RGB format with the channel as the _innermost_ dimension, we are arranging for the needed transformations here.

In [11]:
# load the mean ImageNet image (as distributed with Caffe) for subtraction
mu = np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy')
mu = mu.mean(1).mean(1)  # average over pixels to obtain the mean (BGR) pixel values
print('mean-subtracted values:', zip('BGR', mu))

# create transformer for the input called 'data'
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})

transformer.set_transpose('data', (2,0,1))  # move image channels to outermost dimension
transformer.set_mean('data', mu)            # subtract the dataset-mean value in each channel
transformer.set_raw_scale('data', 255)      # rescale from [0, 1] to [0, 255]
transformer.set_channel_swap('data', (2,1,0))  # swap channels from RGB to BGR

mean-subtracted values: <zip object at 0x1337a3648>


In [12]:
# set the size of the input (we can skip this if we're happy
#  with the default; we can also change it later, e.g., for different batch sizes)
net.blobs['data'].reshape(1,        # batch size
                          3,         # 3-channel (BGR) images
                          227, 227)  # image size is 227x227

### 3. Feature Extraction

In [13]:
from os import listdir
from os.path import isfile, join
name_list = [f for f in listdir(inputpath) if isfile(join(inputpath, f)) and f.endswith('.jpg')]
name_list_test = [f for f in listdir(inputpath_test) if isfile(join(inputpath_test, f)) and f.endswith('.jpg')]

* Layer fc7 dimensions: (1, 4096, 1, 1) [4096 values]

In [15]:
a = datetime.datetime.now()
i = 0
image = caffe.io.load_image(str(inputpath + name_list[i]))
net.blobs['data'].data[...] = transformer.preprocess('data', image)
net.forward()
feature8 = np.reshape(net.blobs['fc7'].data[0], 4096, order='C')
for name in name_list[1:]:
    image = caffe.io.load_image(str(inputpath + name_list[i]))
    net.blobs['data'].data[...] = transformer.preprocess('data', image)
    net.forward()
    feature8 = np.vstack([feature8,np.reshape(net.blobs['fc7'].data[0], 4096, order='C')])
    i += 1
b = datetime.datetime.now()
print(b-a)
print('Feature extraction finished.')

0:06:59.561504
Feature extraction finished.


In [16]:
a1 = datetime.datetime.now()
i = 0
image = caffe.io.load_image(str(inputpath_test + name_list_test[i]))
net.blobs['data'].data[...] = transformer.preprocess('data', image)
net.forward()
feature8_test = np.reshape(net.blobs['fc7'].data[0], 4096, order='C')
for name in name_list_test[1:]:
    image = caffe.io.load_image(str(inputpath_test + name_list_test[i]))
    net.blobs['data'].data[...] = transformer.preprocess('data', image)
    net.forward()
    feature8_test = np.vstack([feature8_test,np.reshape(net.blobs['fc7'].data[0], 4096, order='C')])
    i += 1
b1 = datetime.datetime.now()
print(b1-a1)
print('Feature extraction finished.')

0:08:28.988008
Feature extraction finished.


In [17]:
data_fc7 = np.column_stack([pd.DataFrame(name_list),feature8])

In [18]:
a2 = datetime.datetime.now()
test_fc7 = np.column_stack([pd.DataFrame(name_list_test),feature8_test])
b2 = datetime.datetime.now()
print(b2-a2)

0:00:00.138026


### 3*. Feature Decomposition

In [19]:
from sklearn.decomposition import PCA  
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier

In [20]:
data_fc7 = pd.DataFrame(data_fc7).drop(0, axis=1)
test_fc7 = pd.DataFrame(test_fc7).drop(0, axis=1)

pca = PCA(n_components=1000)
pca.fit(data_fc7)
print("Variance percentage_Training (fc7):" )
print(pca.explained_variance_ratio_.sum())

Variance percentage_Training (fc7):
0.958560323765


In [21]:
a3 = datetime.datetime.now()
test_fc7_pca = pca.transform(test_fc7)
pd.DataFrame(test_fc7_pca).to_csv(outputpath + 'test_fc7_pca.csv')
b3 = datetime.datetime.now()
print(b3-a3)

0:00:03.968979


In [26]:
print("Total time spent (fc7):")
print((b1-a1)+(b2-a2)+(b3-a3))

Total time spent (fc7):
0:08:33.095013
